In [1]:
from iteru import *

In [2]:
aoi = GERD_aoi_dam

In [34]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filterDate('2021-01-01','2022-01-01')\
    .select(['VV','VH'])

In [35]:
dates = get_imgCol_dates(SAR)

In [36]:
def filterSpeckles(img):
    
    VV_smooth = ee.Image(toDB(RefinedLee(toNatural(SAR.select(['VV']))))).rename('VV_Filtered')
    
    return img.addBands(VV_smooth)

In [37]:
SAR = SAR.map(filterSpeckles)

AttributeError: 'ImageCollection' object has no attribute 'divide'

In [ ]:
# def addOtsuThreshold(img):
    
#     histogram = img.select('VV_Filtered').reduceRegion(               
#             reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
#             geometry = aoi, 
#             scale = 10,
#             bestEffort = True  
#                               )
#     otsu_threshold = otsu(histogram.get('VV_Filtered_histogram'))
    
#     water_mask = img.select('VV_Filtered').lt(ee.Number(otsu_threshold).subtract(ee.Number(1))).selfMask().rename('water_mask')
    
    
#     return img.addBands(water_mask).set({"otsu_threshold": otsu_threshold})

In [ ]:
# SAR = SAR.map(addOtsuThreshold)

In [ ]:
def water_classify_threshold(col, threshold):
    
    
    def water_classify(img):
        
        VV_Filtered  = img.select('VV_Filtered')
        water = VV_Filtered.lt(threshold).rename('water')
        water_mask = water.updateMask(water).rename('water_mask')
        
        return img.addBands([water_mask])
    
    return col.map(water_classify)

In [ ]:
SAR = water_classify_threshold(SAR, threshold = -18)

In [ ]:
def calc_area(feature):
    
    area = feature.geometry().area(maxError = 1)
    
    return feature.set({'Area':area})

In [ ]:
def water_to_vector(img):
    
    water_mask = img.select('water_mask').clip(aoi)
    
    feature = ee.Image(1).updateMask(water_mask).reduceToVectors(
           geometry = water_mask.geometry(),
           crs ='EPSG:32636',
           scale = 10 , 
           geometryType = 'polygon',
           eightConnected = False,
           labelProperty =  'water_cover',
           bestEffort = True
          )
    feature = feature.map(calc_area)
    
    lake_feature = feature.sort('Area', False).first()
    
    lake_area = lake_feature.geometry().area(maxError = 1) 
    
    return ee.FeatureCollection(lake_feature).set({'Area':lake_area})\
           .copyProperties(img, img.propertyNames())\
           .copyProperties(lake_feature, lake_feature.propertyNames())

In [ ]:
water_vectors = SAR.map(water_to_vector)

In [ ]:
water_area = [area /(1e6) for area in water_vectors.aggregate_array('Area').getInfo()]

In [ ]:
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(7)
plt.xlabel('Sentinel-1 Image date',fontsize=18)
plt.ylabel('Water Surface Area (km^2)',fontsize=20)
plt.grid(True)
plt.xticks(rotation=90)
plt.plot(dates,water_area,label ='Water Surface Area')
plt.legend()
plt.gcf().set_size_inches(18.5,10.5)

In [ ]:
def max_water_ele(feature):
    
    lake_dem = elevation_dataset.clip(feature)
    
    max_ele = lake_dem.reduceRegion(
                         reducer = ee.Reducer.max(),
                         geometry = lake_dem.geometry(), 
                         scale = 30, 
                         crs = 'EPSG:32636',
                        maxPixels = 1e11).get('DSM')
    return lake_dem.set({'Maximum_water_elevation':max_ele,'Image_date':feature.get('start_day')})

In [ ]:
dems = water_vectors.map(max_water_ele)

In [ ]:
max_elev = [elev for elev in dems.aggregate_array('Maximum_water_elevation').getInfo()]

In [ ]:
plt.style.use('fivethirtyeight')
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(7)
plt.xlabel('Sentinel-1 Image date',fontsize=18)
plt.ylabel('Water Surface Area (km^2)',fontsize=20)
plt.grid(True)
plt.xticks(rotation=90)
plt.plot(dates,max_elev,label ='Water Surface Area')
plt.legend()
plt.gcf().set_size_inches(18.5,10.5)

In [ ]:
def water_vol(lake_dem):
    
    elevations = lake_dem.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=lake_dem.geometry(),
        maxPixels=1e11,
        scale=30,
        crs = 'EPSG:32636',
        bestEffort = True
    ).get('DSM')
    
    elev_pixles_num = ee.List(elevations).length()
    
    elev_sum = ee.List(elevations).reduce(ee.Reducer.sum())
    
    stats = {'Pixels_number':elev_pixles_num,
             'Elevation_sum': elev_sum
            }
    
    return ee.Feature(None,stats)

In [ ]:
volume_stats = dems.map(water_vol)

In [ ]:
ele_sum = volume_stats.aggregate_array('Elevation_sum').getInfo()

In [ ]:
pixel_num  = volume_stats.aggregate_array('Pixels_number').getInfo()

In [ ]:
volume = [((water_level*pixles_count-elevations_sum)*900)/(1e9) for water_level,elevations_sum , pixles_count in zip(max_elev,ele_sum,pixel_num)]
    

In [ ]:
plt.style.use('fivethirtyeight')
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(7)
plt.xlabel('Sentinel-1 Image date',fontsize=18)
plt.ylabel('Water Surface Area (km^2)',fontsize=20)
plt.grid(True)
plt.xticks(rotation=90)
plt.plot(dates,volume,label ='Water Surface Area')
plt.legend()
plt.gcf().set_size_inches(18.5,10.5)